In [193]:
import random
import numpy as np
#import igraph
import csv
from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn import preprocessing
import nltk
import networkx as nx
from sklearn.tree import DecisionTreeRegressor

nltk.download('punkt') # for tokenization
nltk.download('stopwords')
stpwds = set(nltk.corpus.stopwords.words("english"))
stemmer = nltk.stem.PorterStemmer()

#with open("testing_set.txt", "r") as f:
#    reader = csv.reader(f)
#    testing_set  = list(reader)

#testing_set = [element[0].split(" ") for element in testing_set]

[nltk_data] Downloading package punkt to /Users/collinli/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/collinli/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [130]:
#calcualte inner product
def inner(X,Y):
    s = 0
    for i in range(0,len(X)):
        s = s + X[i]*Y[i]
    return s

In [131]:
with open("amazon-meta_item_item_0.txt", "r") as f:
    reader = csv.reader(f)
    training_set  = list(reader)

training_set = [element[0].split(" ") for element in training_set]

with open("amazon-meta_item_info.csv", "r") as f:
    reader = csv.reader(f,delimiter='^')
    node_info  = list(reader)

node_info = node_info[1:]
IDs = [element[0] for element in node_info]

# compute TFIDF vector of each paper
title = [element[1] for element in node_info]
vectorizer = TfidfVectorizer(stop_words="english")
# each row is a node in the order of node_info
features_TFIDF = vectorizer.fit_transform(title)

In [132]:
with open("amazon-meta_filtered_ground_truth.txt", "r") as f:
    reader = csv.reader(f)
    testing_set  = list(reader)

testing_set = [element[0].split(" ")[0] for element in testing_set]

In [133]:
testing_list = []
for i in xrange(len(testing_set)):
    source = testing_set[i]
    for j in xrange(len(testing_set)):
        target = testing_set[j]
        testing_list.append([source,target])

In [134]:
to_keep = random.sample(range(len(training_set)), k=int(round(len(training_set))))
training_set_reduced_total = [training_set[i] for i in to_keep]
#create a local test set
training_set_reduced = training_set_reduced_total[:len(training_set_reduced_total)/2]
testing_set = training_set_reduced_total[len(training_set_reduced_total)/2:]
print(len(training_set_reduced))
print(len(testing_set))
#training_set_reduced = training_set_reduced_total

107416
107416


In [135]:
#read in graph, each node represents a paper
training_set_reduced_total = [training_set[i] for i in to_keep]
G = nx.Graph()
for t in training_set_reduced_total:
    G.add_node(t[0])
    G.add_node(t[1])
    if t[2] == '1':
        G.add_edge(t[0],t[1])

In [136]:
# title TFIDF cos similarity
title_similarity = []

counter = 0
for i in xrange(len(training_set_reduced)):
#for i in range(1):
    source = training_set_reduced[i][0]
    target = training_set_reduced[i][1]
    
    index_source = IDs.index(source)
    index_target = IDs.index(target)
    
    source_info = [element for element in node_info if element[0]==source][0]
    target_info = [element for element in node_info if element[0]==target][0]
    
	# calculate title TFIDF cos similarity
    source_title_tfidf = features_TFIDF[index_source].toarray()[0]
    target_title_tfidf = features_TFIDF[index_target].toarray()[0]
    
    cos = np.dot(source_title_tfidf,target_title_tfidf) / (np.linalg.norm(np.array(source_title_tfidf))\
       *np.linalg.norm(np.array(target_title_tfidf)))
    title_similarity.append(cos)
    
    counter += 1
    if counter % 10000 == True:
        print counter, "training examples processsed"

1 training examples processsed
10001 training examples processsed
20001 training examples processsed
30001 training examples processsed
40001 training examples processsed
50001 training examples processsed
60001 training examples processsed
70001 training examples processsed
80001 training examples processsed
90001 training examples processsed
100001 training examples processsed


In [137]:
# group is the same or not
group_similarity = []

counter = 0
for i in xrange(len(training_set_reduced)):
#for i in range(1):
    source = training_set_reduced[i][0]
    target = training_set_reduced[i][1]
    
    index_source = IDs.index(source)
    index_target = IDs.index(target)
    
    source_info = [element for element in node_info if element[0]==source][0]
    target_info = [element for element in node_info if element[0]==target][0]
    
    #group similarity true or false
    group_source = source_info[2]
    group_target = target_info[2]
    
    if group_source == group_target:
        group_similarity.append(1)
    else:
        group_similarity.append(0)
        
    counter += 1
    if counter % 10000 == True:
        print counter, "training examples processsed"

1 training examples processsed
10001 training examples processsed
20001 training examples processsed
30001 training examples processsed
40001 training examples processsed
50001 training examples processsed
60001 training examples processsed
70001 training examples processsed
80001 training examples processsed
90001 training examples processsed
100001 training examples processsed


In [138]:
# number of reviews
reviews_num_similarity = []

counter = 0
for i in xrange(len(training_set_reduced)):
#for i in range(1):
    source = training_set_reduced[i][0]
    target = training_set_reduced[i][1]
    
    index_source = IDs.index(source)
    index_target = IDs.index(target)
    
    source_info = [element for element in node_info if element[0]==source][0]
    target_info = [element for element in node_info if element[0]==target][0]
    
    # number of reviews similarity
    review_source = source_info[4]
    review_target = target_info[4]
    
    reviews_num_similarity.append(np.absolute(int(review_source)-int(review_target)))
    
    counter += 1
    if counter % 10000 == True:
        print counter, "training examples processsed"

1 training examples processsed
10001 training examples processsed
20001 training examples processsed
30001 training examples processsed
40001 training examples processsed
50001 training examples processsed
60001 training examples processsed
70001 training examples processsed
80001 training examples processsed
90001 training examples processsed
100001 training examples processsed


In [139]:
# same number of words in detailed category
category_similarity = []

counter = 0
for i in xrange(len(training_set_reduced)):
#for i in range(1):
    source = training_set_reduced[i][0]
    target = training_set_reduced[i][1]
    
    index_source = IDs.index(source)
    index_target = IDs.index(target)
    
    source_info = [element for element in node_info if element[0]==source][0]
    target_info = [element for element in node_info if element[0]==target][0]
    
    # same number of words in detailed category
    category_source = source_info[6+int(review_source)*4:]
    category_target = target_info[6+int(review_target)*4:]
    
    category_similarity.append(len(set(category_source).intersection(set(category_target))))
    
    counter += 1
    if counter % 10000 == True:
        print counter, "training examples processsed"

1 training examples processsed
10001 training examples processsed
20001 training examples processsed
30001 training examples processsed
40001 training examples processsed
50001 training examples processsed
60001 training examples processsed
70001 training examples processsed
80001 training examples processsed
90001 training examples processsed
100001 training examples processsed


In [140]:
# rating 
rating_similarity = []

counter = 0
for i in xrange(len(training_set_reduced)):
#for i in range(1):
    source = training_set_reduced[i][0]
    target = training_set_reduced[i][1]
    
    index_source = IDs.index(source)
    index_target = IDs.index(target)
    
    source_info = [element for element in node_info if element[0]==source][0]
    target_info = [element for element in node_info if element[0]==target][0]
    
    # rating similarity
    rating_source = source_info[5]
    rating_target = target_info[5]
    
    rating_similarity.append(np.absolute(float(rating_source)-float(rating_target)))
    
    counter += 1
    if counter % 10000 == True:
        print counter, "training examples processsed"

1 training examples processsed
10001 training examples processsed
20001 training examples processsed
30001 training examples processsed
40001 training examples processsed
50001 training examples processsed
60001 training examples processsed
70001 training examples processsed
80001 training examples processsed
90001 training examples processsed
100001 training examples processsed


In [141]:
# sum of nodes' degrees 
node_degree = []

counter = 0
for i in xrange(len(training_set_reduced)):
    source = training_set_reduced[i][0]
    target = training_set_reduced[i][1]
    
    #add node degree
    source_degree = G.degree(source)
    target_degree = G.degree(target)
    if isinstance(source_degree,dict):
        source_degree = 0
    if isinstance(target_degree,dict):
        target_degree = 0
    node_degree.append(source_degree+target_degree)

In [168]:
# distance between two nodes in graph
dist_node = []

counter = 0
node_set = G.nodes()
for i in xrange(len(training_set_reduced)):
    source = training_set_reduced[i][0]
    target = training_set_reduced[i][1]
    if (source in node_set) and (target in node_set) and nx.has_path(G,source,target):
        dist_node.append(-nx.shortest_path_length(G,source,target))
    else:
        dist_node.append(0)

In [164]:
# number of common neighbors
common_neighbors = []

counter = 0
for i in xrange(len(training_set_reduced)):
#for i in range(1):
    source = training_set_reduced[i][0]
    target = training_set_reduced[i][1]
    
    #add node degree
    common_neighbors.append(len(list(nx.common_neighbors(G,source,target))))
    counter += 1
    if counter % 10000 == True:
        print counter, "training examples processsed"

1 training examples processsed
10001 training examples processsed
20001 training examples processsed
30001 training examples processsed
40001 training examples processsed
50001 training examples processsed
60001 training examples processsed
70001 training examples processsed
80001 training examples processsed
90001 training examples processsed
100001 training examples processsed


In [183]:
# compute betweenness centralities for each node
# since this is a very slow computation, for each node i , we pick 1600 (5% of len(training_set_reduced) 
# other nodes to compute the betweenness centrality of i.
all_btw_centralities = nx.betweenness_centrality(G)

In [184]:
# difference between betweenness-centralities of target and source
# idea comes from https://www.researchgate.net/publication/220432611_Link_Prediction_in_Citation_Networks, page 80-81
btw_centralities = []

counter = 0
node_set = G.nodes()

for i in xrange(len(training_set_reduced)):
    source = training_set_reduced[i][0]
    target = training_set_reduced[i][1]
    if (source in all_btw_centralities) and (target in all_btw_centralities):
        btw_centralities.append(all_btw_centralities[target] - all_btw_centralities[source])
    else:
        btw_centralities.append(0.0)    

In [197]:
# convert list of lists into array
# documents as rows, unique words as columns (i.e., example as rows, features as columns)
training_features = np.array([title_similarity, group_similarity, category_similarity,reviews_num_similarity,rating_similarity,node_degree,dist_node,common_neighbors]).T
#training_features = np.array([overlap_title_tm, temp_diff_tm, comm_auth_tm, text_mining]).T

# scale
training_features = preprocessing.scale(training_features)

In [143]:
# convert labels into integers then into column array
labels = [int(element[2]) for element in training_set_reduced]
labels = list(labels)
labels_array = np.array(labels)

In [8]:
#random prediction
predictions_random = np.random.choice([0, 1], size=len(tmp_test_features))

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [186]:
# initialize basic SVM
classifier = svm.LinearSVC(class_weight='balanced')
# train
classifier.fit(training_features, labels_array)

LinearSVC(C=1.0, class_weight='balanced', dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [16]:
# initialize basic SVM
classifier = svm.SVC()
# train
classifier.fit(training_features, labels_array)

KeyboardInterrupt: 

In [198]:
# Decision Tree regressor
classifier = DecisionTreeRegressor(max_depth=2)
classifier.fit(training_features, labels_array)

DecisionTreeRegressor(criterion='mse', max_depth=2, max_features=None,
           max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=False, random_state=None,
           splitter='best')

In [203]:
#linear regression
theta = np.linalg.lstsq(training_features, labels_array)[0]
print theta

[ 0.02869666  0.          0.02202334  0.00493384  0.00290092  0.01875185
 -0.39529756  0.14019384]


In [145]:
# test
# we need to compute the features for the testing set

title_similarity_test = []
group_similarity_test = []
reviews_num_similarity_test = []
category_similarity_test = []
rating_similarity_test = []
   
counter = 0
for i in xrange(len(testing_set)):
#for i in range(50):
    source = testing_set[i][0]
    target = testing_set[i][1]
    
    index_source = IDs.index(source)
    index_target = IDs.index(target)
    
    source_info = [element for element in node_info if element[0]==source][0]
    target_info = [element for element in node_info if element[0]==target][0]
    
	# calculate title TFIDF cos similarity
    source_title_tfidf = features_TFIDF[index_source].toarray()[0]
    target_title_tfidf = features_TFIDF[index_target].toarray()[0]
    
    cos = np.dot(source_title_tfidf,target_title_tfidf) / (np.linalg.norm(np.array(source_title_tfidf))\
       *np.linalg.norm(np.array(target_title_tfidf)))
    title_similarity_test.append(cos)
    
    #group similarity true or false
    group_source = source_info[2]
    group_target = target_info[2]
    
    if group_source == group_target:
        group_similarity_test.append(1)
    else:
        group_similarity_test.append(0)
        
    # number of reviews similarity
    review_source = source_info[4]
    review_target = target_info[4]
    
    reviews_num_similarity_test.append(np.absolute(int(review_source)-int(review_target)))
    
    # same number of words in detailed category
    category_source = source_info[6+int(review_source)*4:]
    category_target = target_info[6+int(review_target)*4:]
    
    category_similarity_test.append(len(set(category_source).intersection(set(category_target))))
    
    # rating similarity
    rating_source = source_info[5]
    rating_target = target_info[5]
    
    rating_similarity_test.append(np.absolute(float(rating_source)-float(rating_target)))
    
    counter += 1
    if counter % 10000 == True:
        print counter, "testing examples processsed"

1 testing examples processsed
10001 testing examples processsed
20001 testing examples processsed
30001 testing examples processsed
40001 testing examples processsed
50001 testing examples processsed
60001 testing examples processsed
70001 testing examples processsed
80001 testing examples processsed
90001 testing examples processsed
100001 testing examples processsed


In [146]:
# sum of nodes' degrees 
node_degree_test = []

counter = 0
for i in xrange(len(testing_set)):
#for i in range(1):
    source = testing_set[i][0]
    target = testing_set[i][1]
    
    #add node degree
    source_degree = G.degree(source)
    target_degree = G.degree(target)
    if isinstance(source_degree,dict):
        source_degree = 0
    if isinstance(target_degree,dict):
        target_degree = 0
    node_degree_test.append(source_degree+target_degree)

In [156]:
# distance between two nodes in graph
dist_node_test = []

counter = 0
node_set = G.nodes()
for i in xrange(len(testing_set)):
    source = testing_set[i][0]
    target = testing_set[i][1]
    if (source in node_set) and (target in node_set) and nx.has_path(G,source,target):
        dist_node_test.append(-nx.shortest_path_length(G,source,target))
    else:
        dist_node_test.append(0)

In [171]:
# number of common neighbors
common_neighbors_test = []

counter = 0
for i in xrange(len(testing_set)):
    source = testing_set[i][0]
    target = testing_set[i][1]
    
    #add node degree
    common_neighbors_test.append(len(list(nx.common_neighbors(G,source,target))))
    counter += 1
    if counter % 10000 == True:
        print counter, "training examples processsed"

1 training examples processsed
10001 training examples processsed
20001 training examples processsed
30001 training examples processsed
40001 training examples processsed
50001 training examples processsed
60001 training examples processsed
70001 training examples processsed
80001 training examples processsed
90001 training examples processsed
100001 training examples processsed


In [187]:
# difference between betweenness-centralities of target and source
# idea comes from https://www.researchgate.net/publication/220432611_Link_Prediction_in_Citation_Networks, page 80-81
btw_centralities_test = []

counter = 0
node_set = G.nodes()

for i in xrange(len(testing_set)):
    source = testing_set[i][0]
    target = testing_set[i][1]
    if (source in all_btw_centralities) and (target in all_btw_centralities):
        btw_centralities_test.append(all_btw_centralities[target] - all_btw_centralities[source])
    else:
        btw_centralities_test.append(0.0)    

In [195]:
# convert list of lists into array
# documents as rows, unique words as columns (i.e., example as rows, features as columns)
testing_features = np.array([title_similarity_test, group_similarity_test, category_similarity_test,reviews_num_similarity_test,rating_similarity_test,node_degree_test,dist_node_test,common_neighbors_test]).T

# scale
testing_features = preprocessing.scale(testing_features)

In [199]:
#prediction SVM
predictions_SVM = list(classifier.predict(testing_features))
#predictions = zip(range(len(testing_set)), predictions_SVM)

In [204]:
#prediction Linear
result = [int(t[2]) for t in testing_set]
predictions_linear = []
for i in xrange(len(testing_features)):
#for i in range(50):
    r = inner(theta,testing_features[i])
    #if r >= 0:
    #    predictions_linear.append(1)
    #else:
    #    predictions_linear.append(0)
    predictions_linear.append(r)
predictions = zip(result, predictions_linear)

In [202]:
result = [int(t[2]) for t in testing_set]
compare = zip(result,predictions_SVM)
accuracy = 0
for c in compare:
    if c[0] == c[1]:
        accuracy += 1
accuracy = accuracy*1.0 / len(compare)
print(accuracy)

1.0


In [201]:
result = [int(t[2]) for t in testing_set]
compare = zip(result,predictions_SVM)
accuracy = 0
for c in compare:
    if c[0] == c[1] and c[0] == 1:
        accuracy += 1
print(accuracy)

851


In [206]:
count_p1 = 0
for p in predictions_SVM:
    if p == 1:
        count_p1 += 1
print(count_p1)

5502


In [42]:
accuracy = 0
for p in predictions:
    if p[0] == 1 and p[1] >= 0:
        accuracy += 1
print(accuracy)

749


In [11]:
# write predictions to .csv file suitable for Kaggle (just make sure to add the column names)

with open("improved_predictions.csv","wb") as pred1:
    csv_out = csv.writer(pred1)
    csv_out.writerow(['id','category'])
    for row in predictions:
        csv_out.writerow(row)